In [ ]:
% Add SPM to the path. In case you are running this locally
% In a matlab prompt, you can type this to find the answer --> spm('dir')
% addpath('/home/remi/matlab/SPM/spm12');

WD = pwd;

% Add all the functions that are in the sub directories that we need for CPP_SPM
addpath(genpath(fullfile(WD, '..', 'src')));
addpath(genpath(fullfile(WD, '..', 'lib')));

% Make sure it is all set up
checkDependencies();

## Download data

We are getting the `Mother of All Experiments` (MoAE) from the SPM website.

It is good a **old**-fashioned block design with an auditory task.

In [ ]:
% URL of the data set to download
URL = 'http://www.fil.ion.ucl.ac.uk/spm/download/data/MoAEpilot/MoAEpilot.bids.zip';

%% Get data
fprintf('%-40s:', 'Downloading dataset...');
urlwrite(URL, 'MoAEpilot.zip');
unzip('MoAEpilot.zip', fullfile(WD, 'output'));

In [ ]:
!tree output/

## Preparing the configuration for the analysis


In [52]:
% task to analyze
cfg.taskName = 'auditory';

% Uncomment the line below to run preprocessing in "native" space.
% - use realign and unwarp
% - don't do normalization
% opt.space = 'individual';

% The directory where the data are located
cfg.dataDir = fullfile(WD, 'output', 'MoAEpilot');
exist(cfg.dataDir, 'dir')

% JSON file defining the GLM we will run and the contrasts we will compute
cfg.model.file = fullfile(WD, '..', 'demos', 'MoAE', 'models', 'model-MoAE_smdl.json');

% specify the result to compute
% contrast for activations
cfg.result.Steps(1) = struct( ...
                           'Level',  'subject', ...
                           'Contrasts', struct( ...
                                               'Name', 'listening', ... 
                                               'Mask', false, ...
                                               'MC', 'FWE', ... FWE, none, FDR
                                               'p', 0.05, ...
                                               'k', 0, ...
                                               'NIDM', false));
% contrasts for deactivations
cfg.result.Steps(1).Contrasts(2) = struct( ...
                                        'Name', 'listening_inf_baseline', ...
                                        'Mask', false, ...
                                        'MC', 'none', ... FWE, none, FDR
                                        'p', 0.01, ...
                                        'k', 0, ...
                                        'NIDM', false);

cfg = checkOptions(cfg);
saveOptions(cfg);

ans =  7
Options saved in: 



In [ ]:
cfg

## Getting a report on the data and preparing the derivatives

In [ ]:
% TODO
reportBIDS(cfg)

In [ ]:
bidsCopyRawFolder(cfg, 1)

In [ ]:
!tree output/

## Slice timing correction

This data set has no information relevant to slice timing so this step will be ignored even if you try to run it.

In [ ]:
bidsSTC(cfg)

## Spatial preprocessing

This will run:

- realign
- coregistration of `func` data to `anat`
- segmentation of `anat`
- skullstripping of `anat` and creation of brain mask in native space
- normalization to MNI will be run unless you have set `opt.space = 'individual'` (`opt.space = 'MNI'` is the defaut).

In [ ]:
% Be patient this one takes a while
bidsSpatialPrepro(cfg)

## Quality control

This relies on [`spmup`](https://github.com/CPernet/spmup)

In [ ]:
% The following will not run fully on Octave for now
bidsResliceTpmToFunc(cfg)

In [ ]:
% The following will not run fully on Octave for now (because of some of the functions used by spmup)
anatomicalQA(cfg)

In [ ]:
% The following will not run fully on Octave for now (because of some of the functions used by spmup)
functionalQA(cfg)

In [ ]:
% Smoothing to apply
FWHM = 6;
bidsSmoothing(FWHM, cfg);

In [ ]:
!tree output/

## BIDS model file

Let's have a look at the JSON file that defines the GLM we will run and the contrast we will compute.

In [ ]:
!cat ../demos/MoAE/models/model-MoAE_smdl.json

## Run subject level GLM



In [ ]:
bidsFFX('specifyAndEstimate', cfg, FWHM);

In [ ]:
!tree output/derivatives/SPM12_CPPL/sub-01/stats

## Compute contrasts

In [55]:
cfg.result.Steps(1).Contrasts(1)
cfg.result.Steps(1).Contrasts(2)

ans =

  scalar structure containing the fields:

    MC = FWE
    Mask = 0
    NIDM = 1
    Name = listening
    k = 0
    p =  0.050000

ans =

  scalar structure containing the fields:

    MC = none
    Mask = 0
    NIDM = 1
    Name = listening_inf_baseline
    k = 0
    p =  0.010000



In [ ]:
bidsFFX('contrasts', cfg, FWHM);

## Prepare results and package them as NIDM-results

In [ ]:
bidsResults(cfg, FWHM);

In [57]:
cfg.result.Steps(1).Contrasts(1).NIDM = true;

In [ ]:
bidsResults(cfg, FWHM);